In [133]:
import sqlite3
import pandas as pd

In [134]:
pd.options.display.max_columns = 99
pd.options.display.max_colwidth = 15
pd.options.display.width = None
pd.options.display.max_rows = 50
pd.options.display.min_rows = 20

In [135]:
pd.describe_option()

compute.use_bottleneck : bool
    Use the bottleneck library to accelerate if it is installed,
    the default is True
    Valid values: False,True
    [default: True] [currently: True]
compute.use_numba : bool
    Use the numba engine option for select operations if it is installed,
    the default is False
    Valid values: False,True
    [default: False] [currently: False]
compute.use_numexpr : bool
    Use the numexpr library to accelerate computation if it is installed,
    the default is True
    Valid values: False,True
    [default: True] [currently: True]
display.chop_threshold : float or None
    if set to a float value, all float values smaller then the given threshold
    will be displayed as exactly 0 by repr and friends.
    [default: None] [currently: None]
display.colheader_justify : 'left'/'right'
    Controls the justification of column headers. used by DataFrameFormatter.
    [default: right] [currently: right]
display.column_space No description available.
    [defa

In [136]:
csv_data = pd.read_csv("output.csv")
pd.to_datetime(csv_data['date_key'])
con = sqlite3.connect(":memory:")

In [137]:
csv_data.to_sql("users", con, if_exists="replace")

86416

In [138]:
# QUERY CONTAINER
query1 = """

WITH table1_temp AS (
SELECT
account_id,
date_key,
revenue,
LEAD(date_key, 1) OVER (PARTITION BY account_id ORDER BY date_key) as lead_1_month,
LEAD(date_key, 3) OVER (PARTITION BY account_id ORDER BY date_key) as lead_3_month,
LEAD(date_key, 6) OVER (PARTITION BY account_id ORDER BY date_key) as lead_6_month,
LEAD(date_key, 12) OVER (PARTITION BY account_id ORDER BY date_key) as lead_12_month
FROM users
WHERE 1=1
AND account_id = '5edf63c9-a4de-4592-b70d-8fa137c00f88'
),

table2_temp AS (
SELECT
account_id,
date_key,
revenue,
ROUND((JULIANDAY(date_key) - JULIANDAY(lead_1_month))/(365/12),0) as lead_1_mo_diff,
ROUND((JULIANDAY(date_key) - JULIANDAY(lead_3_month))/(365/12),0) as lead_3_mo_diff,
ROUND((JULIANDAY(date_key) - JULIANDAY(lead_6_month))/(365/12),0) as lead_6_mo_diff,
ROUND((JULIANDAY(date_key) - JULIANDAY(lead_12_month))/(365/12),0) as lead_12_mo_diff
FROM table1_temp
),

table3_temp AS (
SELECT
date_key,
COUNT(account_id) AS c_cur,
COUNT(CASE WHEN lead_1_mo_diff=-1.0 THEN account_id ELSE NULL END) AS c_1m,
COUNT(CASE WHEN lead_3_mo_diff=-3.0 THEN account_id ELSE NULL END) AS c_3m,
COUNT(CASE WHEN lead_6_mo_diff=-6.0 THEN account_id ELSE NULL END) AS c_6m,
COUNT(CASE WHEN lead_12_mo_diff=-12.0 THEN account_id ELSE NULL END) AS c_12m
FROM table2_temp
GROUP BY 1
)

SELECT date_key, lead_1_month, lead_3_month, lead_6_month, lead_12_month FROM table1_temp

"""

query2 = """
SELECT * FROM users where account_id = '5edf63c9-a4de-4592-b70d-8fa137c00f88'
"""

query3 = """
WITH calendar AS (
SELECT
DISTINCT(date_key)
FROM users
ORDER BY date_key
),

table1_temp AS (
SELECT
account_id,
date_key,
revenue,
LEAD(date_key, 1) OVER (ORDER BY date_key) as lead_1_month,
IFNULL(ROUND((JULIANDAY(date_key) - JULIANDAY(LEAD(date_key, 1) OVER (ORDER BY date_key)))/(365/12),0), 0.0) as lead_1_mo_diff,
LEAD(date_key, 3) OVER (PARTITION BY account_id ORDER BY date_key) as lead_3_month,
IFNULL(ROUND((JULIANDAY(date_key) - JULIANDAY(LEAD(date_key, 3) OVER (PARTITION BY account_id ORDER BY date_key)))/(365/12),0), 0.0) as lead_3_mo_diff,
LEAD(date_key, 6) OVER (PARTITION BY account_id ORDER BY date_key) as lead_6_month,
IFNULL(ROUND((JULIANDAY(date_key) - JULIANDAY(LEAD(date_key, 6) OVER (PARTITION BY account_id ORDER BY date_key)))/(365/12),0), 0.0) as lead_6_mo_diff,
LEAD(date_key, 12) OVER (PARTITION BY account_id ORDER BY date_key) as lead_12_month,
IFNULL(ROUND((JULIANDAY(date_key) - JULIANDAY(LEAD(date_key, 12) OVER (PARTITION BY account_id ORDER BY date_key)))/(365/12),0), 0.0) as lead_12_mo_diff
FROM users
WHERE 1=1
AND account_id IN ('5edf63c9-a4de-4592-b70d-8fa137c00f88', 'b41abbf3-ca43-48cd-8e8d-1aad0729bae6')
AND date_key BETWEEN '2019-01-01' AND '2019-12-31'
),

table2_temp AS (
SELECT
account_id,
date_key,
revenue,
lead_1_month,
lead_1_mo_diff,
SUM(CASE WHEN lead_1_mo_diff = -1.0 THEN 1 ELSE 0 END) as c_1m,
lead_3_month,
lead_3_mo_diff,
SUM(CASE WHEN lead_3_mo_diff = -3.0 THEN 1 ELSE 0 END) as c_3m
--date_key,
--COUNT(account_id) AS c_cur,
--COUNT(CASE WHEN lead_1_mo_diff=-1.0 THEN account_id ELSE NULL END) AS c_1m,
--COUNT(CASE WHEN lead_3_mo_diff=-3.0 THEN account_id ELSE NULL END) AS c_3m,
--COUNT(CASE WHEN lead_6_mo_diff=-6.0 THEN account_id ELSE NULL END) AS c_6m,
--COUNT(CASE WHEN lead_12_mo_diff=-12.0 THEN account_id ELSE NULL END) AS c_12m
FROM table1_temp
GROUP BY
account_id,
date_key,
revenue,
lead_1_month,
lead_1_mo_diff,
lead_3_month,
lead_3_mo_diff
)

-- SELECT * FROM calendar


SELECT
account_id,
date_key,
revenue,
lead_1_month,
lead_1_mo_diff,
--SUM(c_1m),
lead_3_month,
lead_3_mo_diff
--SUM(c_3m)
FROM table1_temp
ORDER BY 
date_key

"""

query4 = """
WITH calendar AS (
SELECT
date_key,
LEAD(date_key, 3) OVER (ORDER BY date_key) as date_key3,
LEAD(date_key, 6) OVER (ORDER BY date_key) as date_key6,
LEAD(date_key, 12) OVER (ORDER BY date_key) as date_key12
FROM users
WHERE 1=1
AND date_key > '2019-06-01'
GROUP BY date_key
ORDER BY date_key
),

fact_table_curr AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
),

fact_table_3m AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
),

fact_table_6m AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
),

fact_table_12m AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
),

all_joined AS (
SELECT
calendar.date_key as c_date_key,
fact_table_curr.date_key as f_curr_date_key,
SUBSTR(fact_table_curr.account_id,1,3) as f_curr_acc_id,
fact_table_curr.revenue as f_curr_rev,
calendar.date_key3 as c_date_key3,
fact_table_3m.date_key as f_3m_date_key,
SUBSTR(fact_table_3m.account_id,1,3) as f_3m_acc_id,
fact_table_6m.revenue as f_6m_rev,
calendar.date_key6 as c_date_key6,
fact_table_6m.date_key as f_6m_date_key,
SUBSTR(fact_table_6m.account_id,1,3) as f_6m_acc_id,
fact_table_6m.revenue as f_6m_rev,
calendar.date_key12 as c_date_key12,
fact_table_12m.date_key as f_12m_date_key,
SUBSTR(fact_table_12m.account_id,1,3) as f_12m_acc_id,
fact_table_12m.revenue as f_12m_rev
FROM calendar
LEFT JOIN fact_table_curr ON calendar.date_key = fact_table_curr.date_key
LEFT JOIN fact_table_3m ON (fact_table_curr.account_id = fact_table_3m.account_id AND calendar.date_key3 = fact_table_3m.date_key)
LEFT JOIN fact_table_6m ON (fact_table_curr.account_id = fact_table_6m.account_id AND calendar.date_key6 = fact_table_6m.date_key)
LEFT JOIN fact_table_12m ON (fact_table_curr.account_id = fact_table_12m.account_id AND calendar.date_key12 = fact_table_12m.date_key)
)

SELECT
c_date_key,
COUNT(f_curr_acc_id) as count_acc_curr,
COUNT(f_3m_acc_id) as count_acc_3m,
COUNT(f_6m_acc_id) as count_acc_6m,
COUNT(f_12m_acc_id) as count_acc_12m
FROM all_joined
WHERE 1=1
GROUP BY
c_date_key
ORDER BY
c_date_key

"""

query4_verif = """
WITH calendar AS (
SELECT
date_key,
LEAD(date_key, 1) OVER () as date_key1,
LEAD(date_key, 3) OVER () as date_key3,
LEAD(date_key, 6) OVER () as date_key6
--LEAD(date_key, 12) OVER () as date_key12
FROM users
WHERE 1=1
AND date_key BETWEEN '2019-01-01' AND '2019-07-01'
GROUP BY date_key
ORDER BY date_key
),

fact_table1 AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
AND date_key BETWEEN '2019-01-01' AND '2019-12-01'
AND account_id IN (
    'eba9dbd2-ff64-46cb-9ab8-08ddcc99485f',
    'c51272a2-28ad-4fa8-80c1-5b7566c439a8',
    '5a316a69-9d5e-4bdc-a908-8a109ed7f4ac',
    '17516fd3-55e1-4ebb-8050-2da61969976e',
    'b2ec20fc-27d1-43cf-83a3-e0ada52d0639'
    )
),

fact_table2 AS (
SELECT
account_id,
date_key,
revenue
FROM users
WHERE 1=1
AND date_key BETWEEN '2019-01-01' AND '2019-12-01'
AND account_id IN (
    'eba9dbd2-ff64-46cb-9ab8-08ddcc99485f',
    'c51272a2-28ad-4fa8-80c1-5b7566c439a8',
    '5a316a69-9d5e-4bdc-a908-8a109ed7f4ac',
    '17516fd3-55e1-4ebb-8050-2da61969976e',
    'b2ec20fc-27d1-43cf-83a3-e0ada52d0639'
    )
),

all_joined AS (
SELECT
calendar.date_key as c_date_key,
fact_table1.date_key as f1_date_key,
SUBSTR(fact_table1.account_id,1,3) as f1_acc_id,
fact_table1.revenue as f1_rev,
calendar.date_key3 as c_date_key3,
fact_table2.date_key as f2_date_key,
SUBSTR(fact_table2.account_id,1,3) as f2_acc_id,
fact_table2.revenue as f2_rev
FROM calendar
LEFT JOIN fact_table1 ON calendar.date_key = fact_table1.date_key
LEFT JOIN fact_table2 ON fact_table1.account_id = fact_table2.account_id AND calendar.date_key3 = fact_table2.date_key
)

SELECT
*
FROM all_joined
WHERE 1=1


"""

query4_presto = """ 

WITH calendar AS (
SELECT
date_key,
LEAD(date_key, 3) OVER (ORDER BY date_key) as date_key3,
LEAD(date_key, 6) OVER (ORDER BY date_key) as date_key6,
LEAD(date_key, 12) OVER (ORDER BY date_key) as date_key12
FROM clv.agg_acc_rev_monthly
WHERE date_key BETWEEN '2019-01-31' AND '2019-06-30'
GROUP BY date_key
ORDER BY date_key
),

fact_table_curr AS (
SELECT
account_id,
date_key,
hr as revenue
FROM clv.agg_acc_rev_monthly
WHERE 1=1
),

fact_table_3m AS (
SELECT
account_id,
date_key,
hr as revenue
FROM clv.agg_acc_rev_monthly
WHERE 1=1
),

fact_table_6m AS (
SELECT
account_id,
date_key,
hr as revenue
FROM clv.agg_acc_rev_monthly
WHERE 1=1
),

fact_table_12m AS (
SELECT
account_id,
date_key,
hr as revenue
FROM clv.agg_acc_rev_monthly
WHERE 1=1
),

all_joined AS (
SELECT
calendar.date_key as c_date_key,
fact_table_curr.date_key as f_curr_date_key,
fact_table_curr.account_id as f_curr_acc_id,
fact_table_curr.revenue as f_curr_rev,
calendar.date_key3 as c_date_key3,
fact_table_3m.date_key as f_3m_date_key,
fact_table_3m.account_id as f_3m_acc_id,
fact_table_3m.revenue as f_3m_rev
--calendar.date_key6 as c_date_key6,
--fact_table_6m.date_key as f_6m_date_key,
--fact_table_6m.account_id as f_6m_acc_id,
--fact_table_6m.revenue as f_6m_rev,
--calendar.date_key12 as c_date_key12,
--fact_table_12m.date_key as f_12m_date_key,
--fact_table_12m.account_id as f_12m_acc_id,
--fact_table_12m.revenue as f_12m_rev
FROM calendar
LEFT JOIN fact_table_curr ON calendar.date_key = fact_table_curr.date_key
LEFT JOIN fact_table_3m ON (fact_table_curr.account_id = fact_table_3m.account_id AND calendar.date_key3 = fact_table_3m.date_key)
--LEFT JOIN fact_table_6m ON (fact_table_curr.account_id = fact_table_6m.account_id AND calendar.date_key6 = fact_table_6m.date_key)
--LEFT JOIN fact_table_12m ON (fact_table_curr.account_id = fact_table_12m.account_id AND calendar.date_key12 = fact_table_12m.date_key)
)

SELECT
c_date_key,
COUNT(f_curr_acc_id) as count_acc_curr,
COUNT(f_3m_acc_id) as count_acc_3m
FROM all_joined
WHERE 1=1
GROUP BY
c_date_key
ORDER BY
c_date_key

"""



In [139]:
q=None
with open(file='users.sql', mode='r') as f:
    q = f.read()

cohort_df = pd.read_sql_query(q, con)
cohort_df.to_sql("users_2", con, index=False, if_exists="replace")
print(cohort_df)


         date_key      account_id   revenue       source
0      2019-01-01  0002335f-26...  0.000000  display_ads
1      2019-02-01  0002335f-26...  0.000000  display_ads
2      2019-03-01  0002335f-26...  0.000000  display_ads
3      2019-04-01  0002335f-26...  0.000000  display_ads
4      2019-05-01  0002335f-26...  0.000000  display_ads
5      2019-06-01  0002335f-26...  0.000000  display_ads
6      2019-07-01  0002335f-26...  7.113665  display_ads
7      2019-08-01  0002335f-26...  5.755132  display_ads
8      2019-09-01  0002335f-26...  3.815103  display_ads
9      2019-10-01  0002335f-26...  0.000000  display_ads
...           ...             ...       ...          ...
71990  2020-03-01  ffc914c7-26...  0.000000  display_ads
71991  2020-04-01  ffc914c7-26...  0.821459  display_ads
71992  2020-05-01  ffc914c7-26...  0.000000  display_ads
71993  2020-06-01  ffc914c7-26...  0.000000  display_ads
71994  2020-07-01  ffc914c7-26...  0.000000  display_ads
71995  2020-08-01  ffc914c7-26.

In [140]:
with open(file='users_2.sql', mode='r') as f:
    q = f.read()

users_df = pd.read_sql_query(q, con)
print(users_df)

      date_key  churned_3m  not_churned_3m
0   2019-02-01           0             692
1   2019-03-01           0             692
2   2019-04-01           0             692
3   2019-05-01         145             547
4   2019-06-01         149             543
5   2019-07-01         136             556
6   2019-08-01         144             548
7   2019-09-01         160             532
8   2019-10-01         162             530
9   2019-11-01         167             525
10  2019-12-01         157             535
11  2020-01-01         171             521
12  2020-02-01         149             543
13  2020-03-01         151             541
14  2020-04-01         166             526
15  2020-05-01         151             541
16  2020-06-01         149             543
17  2020-07-01         149             543
18  2020-08-01         159             533
19  2020-09-01         148             544
20  2020-10-01         160             532
21  2020-11-01         159             533
22  2020-12

In [142]:
with open(file='users_3.sql', mode='r') as f:
    q = f.read()

users_df = pd.read_sql_query(q, con)
print(users_df)
# users_df.to_csv(path_or_buf=f"./users_3.csv", index=True)

            dk  SUM(m6)  SUM(rt)  still_m6  act  inact
0   2019-02-01        0        0       NaN  692      0
1   2019-03-01        0        0       0.0  281    411
2   2019-04-01        0        0       0.0  264    428
3   2019-05-01        0        0       0.0  296    396
4   2019-06-01        0        0       0.0  269    423
5   2019-07-01        0        0       0.0  283    409
6   2019-08-01       29        0       0.0  274    418
7   2019-09-01       38       10      19.0  265    427
8   2019-10-01       39       13      25.0  265    427
9   2019-11-01       36       15      24.0  245    447
10  2019-12-01       41       15      21.0  290    402
11  2020-01-01       47       15      26.0  282    410
12  2020-02-01       35       22      25.0  276    416
13  2020-03-01       39        9      26.0  265    427
14  2020-04-01       40       12      27.0  282    410
15  2020-05-01       30       20      20.0  281    411
16  2020-06-01       29       12      18.0  253    439
17  2020-0